```python
## \file src/webdriver/llm_driver/simple_browser.py
# -*- coding: utf-8 -*-
#! .pyenv/bin/python3
```
Модуль для запуска задач с использованием LLM через LangChain и стандартных агентов.
==================================================================================
(Использует инструменты, взаимодействующие с BrowserController и/или API поиска)

Предоставляет функциональность для:
- Конфигурирования моделей (Gemini, OpenAI).
- Установки API ключей.
- Запуска задачи с использованием LLM и доступных инструментов (веб-поиск, браузер).
- Выполнения задачи до конечного результата (`run_task`).
- Стриминга выполнения задачи (`stream_task`).

Зависимости:
    - langchain-openai, langchain-google-genai, langchain-core, langchainhub, langchain
    - langchain-community (для SerpAPIWrapper)
    - google-search-results (для SerpAPIWrapper)
    - python-dotenv
    - browser_use (или ваш модуль с BrowserController)
    - src.gs, src.logger, src.utils, header
```rst
.. module:: src.webdriver.llm_driver.simple_browser
```

In [ ]:
import ipdb # <- трасировка и точки останова
import os
import asyncio
from types import SimpleNamespace
from typing import List, Dict, Any, Optional, Callable, Type, Tuple, AsyncIterator
from pathlib import Path

# LangChain компоненты
from langchain_openai import ChatOpenAI
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.agents import AgentExecutor, create_react_agent, Tool
from langchain_core.language_models.chat_models import BaseChatModel
from langchain_core.exceptions import LangChainException
from langchain import hub
# --- Инструмент для поиска через API ---
# Убедитесь, что установлена: pip install google-search-results
from langchain_community.utilities import SerpAPIWrapper
from browser_use import Agent

# --- Внутренние модули ---
import header
from header import __root__
from src import gs
# from src.webdriver.ai_browser import tools
# from src.webdriver.ai_browser.tools import get_tools, get_tools_by_type, get_tools_by_name
from src.webdriver.llm_driver.use_llm import Config, Driver, stream_agent_execution

from src.logger import logger
from src.utils.jjson import j_loads, j_loads_ns
from src.utils.printer import pprint as print

from dotenv import load_dotenv
load_dotenv()

In [ ]:
class Config:
    ENDPOINT:Path = Path(__root__/'SANDBOX'/'davidka')

In [ ]:
class SimpleDriver(Driver):
    """"""
    def __init__(self, 
                 GEMINI_API_KEY:str = None, 
                 OPENAI_API_KEY:str = None, 
                 openai_model_name:str = None, 
                 gemini_model_name:str = None, 
                 start_browser:str = True,
                **kwargs):
        super().__init__(GEMINI_API_KEY, OPENAI_API_KEY, openai_model_name, gemini_model_name, start_browser, **kwargs) 

    async def simple_process_task_async(self, task:str = 'Hello, world!') -> Any:
        """"""

        result_dict:dict = {}

        def clean_json(raw_text: str) -> str:
            # 1. Убираем всё до первой фигурной скобки {

            json_start = raw_text.find('{')
            if json_start == -1:
                return raw_text  # нет скобки, вернуть как есть
            # Извлекаем текст начиная с первой фигурной скобки
            json_cleaned = raw_text[json_start:]
    
            # 2. Убираем завершающие тройные кавычки или лишние пробелы
            json_cleaned = json_cleaned.strip('`\n ')
            
            return json_cleaned

        try:
            # Инициализация агента с списком моделей и задачей
            # Убедитесь, что ваш класс Agent может принимать список LLM объектов в параметре 'llm'
            agent = Agent(
                task=task,
                llm=self.gemini, # Передача инициализированнoй модели
                # Другие параметры для Agent, если они есть
            )
            logger.info(f"Агент начинает выполнение задачи: \"{task}\"")
            answer: Any = await agent.run() # Ожидание результата работы агента
            if not answer:
                logger.error('Не вернулся результат действий агента')
                ...
            timestamp:str = gs.now

            for action_result in answer.history:
                result_list:list = getattr(action_result, 'result', None)
                result:'ActionResult' = result_list[0]
                extracted_content:str =	result.extracted_content

                cleaned_json_text = clean_json(extracted_content)
                try:
                    data = j_loads(cleaned_json_text)  # Загружаем JSON из текста
                    if not data: continue
                except Exception as ex:
                    logger.error("Ошибка разбора JSON", ex, exc_info=True)
                    ...
                    continue

                # Сохраняем данные
                timestamp = gs.now
                j_dumps(data, Config.ENDPOINT/'train_data_products'/f'product_links_{timestamp}.json')
                result_dict.update(data)

            logger.info("Агент завершил выполнение задачи.")
            ...
            return result_dict 
        except Exception as agent_err:
            logger.error(f"\n\n !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!/nПроизошла ошибка во время инициализации или выполнения задачи агентом./n!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!/n", agent_err, exc_info=True)
            ...
            return '' # Возврат None при ошибке агента


```python
## \file /sandbox/davidka/crawler_simple_driver.py
# -*- coding: utf-8 -*-
#! .pyenv/bin/python3
```
Модуль для сбора данных со страниц товаров через SimpleDriver
=====================================================
(адаптация исходного crawler.py)
```rst
.. module:: sandbox.davidka.crawler_simple_driver
```


In [ ]:
import asyncio
import random
from pathlib import Path

import header
from header import __root__
from src import gs
from src.webdriver.llm_driver.simple_driver import SimpleDriver
from src.utils.jjson import j_loads, j_dumps
from src.utils.file import read_text_file, save_text_file, get_filenames_from_directory
from src.utils.url import get_domain
from src.utils.string.ai_string_utils import normalize_answer
from src.utils.printer import pprint as print
from src.logger import logger

In [ ]:
class Config:
    ENDPOINT: Path = __root__ / 'SANDBOX' / 'davidka'
    mining_data_path: Path = ENDPOINT / 'random_urls'
    train_data_supplier_categories_path: Path = ENDPOINT / 'train_data_supplier_categories'
    checked_domains: list = read_text_file(ENDPOINT / 'checked_domains.txt', as_list=True)
    crawl_files_list: list = get_filenames_from_directory(mining_data_path, 'json')
    instruction_grab_product_page_simple_driver: str = (ENDPOINT / 'instructions' / 'grab_product_page_simple_driver.md').read_text(encoding='utf-8')
    instruction_get_supplier_categories: str = (ENDPOINT / 'instructions' / 'get_supplier_categories.md').read_text(encoding='utf-8')
    instruction_find_product_in_supplier_domain: str = (ENDPOINT / 'instructions' / 'find_product_in_supplier_domain.md').read_text(encoding='utf-8')
    instruction_for_products_urls_one_product: str = (ENDPOINT / 'instructions' / 'get_product_links_one_product.md').read_text(encoding='utf-8')
    instruction_links_from_search: str = (ENDPOINT / 'instructions' / 'links_from_search.md').read_text(encoding='utf-8')
    instruction_links_from_searh_page: str = (ENDPOINT / 'instructions' / 'links_from_searh_page.md').read_text(encoding='utf-8')
    GEMINI_API_KEY = gs.credentials.gemini.katia.api_key
    driver: SimpleDriver = SimpleDriver(gemini_model_name='gemini-1.5-flash-8b-exp-0924', GEMINI_API_KEY = GEMINI_API_KEY)



In [ ]:
def get_products_urls_list_from_files(crawl_files_list: list = None) -> list:
    """Читает файлы с товарами и возвращает product_url списком"""
    products_urls_list = []
    for filename in crawl_files_list or Config.crawl_files_list:
        try:
            file_path = Config.mining_data_path / filename
            crawl_data = j_loads(file_path)['products']
            for product in crawl_data:
                products_urls_list.append(product['product_url'])
        except Exception as ex:
            logger.error(f'Ошибка при обработке файла {filename=}', ex, exc_info=True)
    random.shuffle(products_urls_list)
    return products_urls_list


def yield_product_urls_from_files(directory: Path = Config.mining_data_path, pattern: str = 'json'):
    """Генератор url товаров из файлов"""
    filenames = get_filenames_from_directory(directory, pattern)
    for filename in filenames:
        try:
            file_path = directory / filename
            crawl_data = j_loads(file_path)['products']
            for product in crawl_data:
                yield product['product_url']
        except Exception as ex:
            logger.error(f'Ошибка при обработке файла {filename=}', ex, exc_info=True)


def get_categories_from_random_urls(crawl_files_list: list = None) -> list:
    """Возвращает все категории из файлов товаров"""
    categories_list = []
    for filename in crawl_files_list or Config.crawl_files_list:
        try:
            file_path = Config.mining_data_path / filename
            crawl_data = j_loads(file_path)
            crawl_data = crawl_data.get('products', [])
            for product in crawl_data:
                if 'parent_category' in product:
                    categories_list.append(product['parent_category'])
                if 'category_name' in product:
                    categories_list.append(product['category_name'])
        except Exception as ex:
            logger.error(f'Ошибка при обработке файла {filename=}', ex, exc_info=True)
    categories_list = list(filter(None, set(categories_list)))
    random.shuffle(categories_list)
    return categories_list


async def build_random_products_urls_by_category(category: str, task:str = '', num_of_links: str = '10') -> str:
    """Получить товары по категории"""
    try:
        driver = Config.driver
        logger.info(f'Обработка {category=}')
        
        task = task or Config.instruction_links_from_searh_page.replace('{PRODUCT_CATEGORY}', category).replace('{NUM_LINKS}', num_of_links)
        #ipdb.set_trace()
        answer = await driver.simple_process_task_async(task)
        if not answer:
            return ''
        print('\n -------------------------------- EXTRACTED DATA \n------------------------------------------\n')
        print(answer)
        print('\n -------------------------------------------------------------------------------------------')
        save_text_file(answer, Path(f'F:/llm/random_products_links/{gs.now}.json'))
        return answer
    except Exception as ex:
        logger.error(f'Ошибка при обработке {category=}', ex, exc_info=True)
        return ''


async def fetch_categories_from_suppliers_random_urls() -> dict:
    """Сбор категорий с сайтов"""
    categories_dict = {}
    driver = Config.driver

    for filename in Config.crawl_files_list:
        try:
            file_path = Config.mining_data_path / filename
            crawl_data = j_loads(file_path)
            crawl_data = crawl_data.get('products', [])
            for product in crawl_data:
                domain = get_domain(product['product_url'])
                if domain in Config.checked_domains:
                    continue
                task = Config.instruction_get_supplier_categories.replace('{INPUT_URL}', domain)
                res = await driver.simple_process_task_async(task)
                if not res:
                    continue
                normalized_res = normalize_answer(res.get('output', ''))
                data = j_loads(normalized_res)
                print(data)
                j_dumps(data, Config.train_data_supplier_categories_path / f'{gs.now}.json')
                Config.checked_domains.append(domain)
                save_text_file(Config.checked_domains, Config.ENDPOINT / 'checked_domains.txt')
                j_dumps(Config.checked_domains, Config.ENDPOINT / 'checked_domains.json')
        except Exception as ex:
            logger.error(f'Ошибка при обработке файла {filename=}', ex, exc_info=True)
    return categories_dict

In [ ]:
#driver = Config.driver

# Пример: обработка товаров по категориям
for category in get_categories_from_random_urls():
    #ipdb.set_trace()
    if not await build_random_products_urls_by_category(category = category, num_of_links = '10'):
        break

2025-05-04 02:45:14,227 - WARNING - Failed to parse model output:  Expecting value: line 1 column 1 (char 0)
2025-05-04 02:45:14,227 - WARNING - Failed to parse model output:  Could not parse response.
2025-05-04 02:45:14,228 - ERROR - ❌ Result failed 1/3 times:
 Could not parse response.
2025-05-04 02:45:14,265 - INFO - 📍 Step 6
2025-05-04 02:45:21,819 - INFO - 👍 Eval: Success - Successfully extracted links from the search results page.
2025-05-04 02:45:21,819 - INFO - 🧠 Memory: Navigated to Google, performed the search for 'Medium Voltage Switchgear', scrolled down, and extracted the links. I have filtered the links to remove social media and video hosting sites. The task is complete.
2025-05-04 02:45:21,819 - INFO - 🎯 Next goal: Provide the filtered links in the specified JSON format and mark the task as done.
2025-05-04 02:45:21,819 - INFO - 🛠️  Action 1/1: {"done":{"text":"{\n  \"links\": [\n    \"https://www.eaton.com/en-us/products\",\n    \"https://www.eaton.com/en-us/products\


 -------------------------------- EXTRACTED DATA 
------------------------------------------

{
    "data": null,
    "links": [
        "https://www.eaton.com/en-us/products",
        "https://www.eaton.com/en-us/products",
        "https://ecsksa.com/blog/lv-mv-hv-switchgears",
        "https://ecsksa.com/blog/lv-mv-hv-switchgears",
        "https://www.gwelectric.com/blog/2024/11/22/how-...",
        "https://www.gwelectric.com/blog/2024/11/22/how-...",
        "https://www.shentongroup.co.uk/blog/what-is-low-vo...",
        "https://www.shentongroup.co.uk/blog/what-is-low-vo...",
        "https://new.abb.com/medium-voltage",
        "https://new.abb.com/medium-voltage",
        "https://www.ormazabal.com/en-gb/wh...",
        "https://www.ormazabal.com/en-gb/wh...",
        "https://aktif.net/what-is-a-medium-voltag...",
        "https://aktif.net/what-is-a-medium-voltag...",
        "https://www.se.com/product-category/8...",
        "https://www.se.com/product-category/8...",
  

2025-05-04 02:45:23,649 - INFO - 🚀 Starting task: 
```md
**Роль:** Ты — Автоматизированный Веб-Агент для Поиска Страниц.

**Цель:**  
Найти **все ссылки** из первой страницы поиска по запросу `Heat Shrink Tubing (Adhesive Lined)`.

**Инструкция:**

1. **Открой поисковую систему** (например, Google, Bing или аналогичную).

2. 
- **Введи запрос**: `Heat Shrink Tubing (Adhesive Lined)`.

3. **Из результатов поиска**:

   - Собери **все ссылки** на первой странице поиска.
   - Пропускай рекламные и спонсорские результаты.
   - Пропускай ссылки на социальные сети и видеохостинги.

4. **Фильтрация ссылок**:
   - Игнорируй все ссылки, в домене которых содержатся следующие ключевые слова:
     - `youtube`
     - `tiktok`
     - `facebook`
     - `instagram`
     - `twitter`
     - `linkedin`
     - `pinterest`
     - `vk`
     - `reddit`
     - `snapchat`
     - и аналогичные платформы социальных сетей или видеохостингов.
   - Проверку выполняй по части URL (поддомен и основной домен).
   - Ес


 -------------------------------- EXTRACTED DATA 
------------------------------------------

{
    "data": "\ud83d\uddb1\ufe0f  Clicked button with index 20: ",
    "organic_search_urls": [
        "https://www.amazon.com/adhesive-heat-shrink-tubing/s?k=adhesive+heat+shrink+tubing",
        "https://il.farnell.com/c/cable-wire-cable-assemblies/cable-management/heat-shrink/adhesive-lined-heat-shrink-tubing?srsltid=AfmBOor5Nw8Nj7bnt7dNOugceZfgnuBjmVPets3Z4DOpm7v0GuIpC5ri",
        "https://www.mouser.co.il/c/wire-cable/wire-cable-management/heat-shrink-tubing-sleeves/?color=Multicolor&type=Adhesive%20Lined%20%2F%20Dual%20Wall&pop=19nl",
        "https://buyheatshrink.com/3-1-adhesive-lined-heat-shrink-tubing?srsltid=AfmBOoolKNnigMDKntHRZS5v9B5d1Wnat10X6IDrLq4q6hqV-t4lcn47",
        "https://www.te.com/en/products/heat-shrink-tubing/dual-wall-tubing.html",
        "https://www.hellermanntyton.com/competences/heat-shrink-tubing",
        "https://www.grainger.com/category/electrical/wire

2025-05-04 02:46:05,588 - INFO - 🚀 Starting task: 
```md
**Роль:** Ты — Автоматизированный Веб-Агент для Поиска Страниц.

**Цель:**  
Найти **все ссылки** из первой страницы поиска по запросу `Temperature Sensors (Clamp-on)`.

**Инструкция:**

1. **Открой поисковую систему** (например, Google, Bing или аналогичную).

2. 
- **Введи запрос**: `Temperature Sensors (Clamp-on)`.

3. **Из результатов поиска**:

   - Собери **все ссылки** на первой странице поиска.
   - Пропускай рекламные и спонсорские результаты.
   - Пропускай ссылки на социальные сети и видеохостинги.

4. **Фильтрация ссылок**:
   - Игнорируй все ссылки, в домене которых содержатся следующие ключевые слова:
     - `youtube`
     - `tiktok`
     - `facebook`
     - `instagram`
     - `twitter`
     - `linkedin`
     - `pinterest`
     - `vk`
     - `reddit`
     - `snapchat`
     - и аналогичные платформы социальных сетей или видеохостингов.
   - Проверку выполняй по части URL (поддомен и основной домен).
   - Если URL сод

...

2025-05-04 03:02:33,940 - INFO - 🚀 Starting task: 
```md
**Роль:** Ты — Автоматизированный Веб-Агент для Поиска Страниц.

**Цель:**  
Найти **все ссылки** из первой страницы поиска по запросу `SiC FETs`.

**Инструкция:**

1. **Открой поисковую систему** (например, Google, Bing или аналогичную).

2. 
- **Введи запрос**: `SiC FETs`.

3. **Из результатов поиска**:

   - Собери **все ссылки** на первой странице поиска.
   - Пропускай рекламные и спонсорские результаты.
   - Пропускай ссылки на социальные сети и видеохостинги.

4. **Фильтрация ссылок**:
   - Игнорируй все ссылки, в домене которых содержатся следующие ключевые слова:
     - `youtube`
     - `tiktok`
     - `facebook`
     - `instagram`
     - `twitter`
     - `linkedin`
     - `pinterest`
     - `vk`
     - `reddit`
     - `snapchat`
     - и аналогичные платформы социальных сетей или видеохостингов.
   - Проверку выполняй по части URL (поддомен и основной домен).
   - Если URL содержит хотя бы одно из этих слов — пропускай 


 -------------------------------- EXTRACTED DATA 
------------------------------------------

{
    "data": "\ud83d\udd0d  Scrolled down the page by 1000 pixels",
    "search_result_links": [
        "https://www.infineon.com/cms/en/product/power/mosfet/silicon-carbide/",
        "https://translate.google.com/translate?u=https://www.infineon.com/cms/en/product/power/mosfet/silicon-carbide/&hl=iw&sl=en&tl=iw&client=search",
        "https://www.infineon.com/cms/en/product/power/mosfet/silicon-carbide/discretes/",
        "https://www.infineon.com/cms/en/product/power/mosfet/silicon-carbide/1200v/",
        "https://www.infineon.com/cms/en/product/power/mosfet/silicon-carbide/modules/",
        "https://www.infineon.com/cms/en/product/power/mosfet/silicon-carbide/400v/",
        "https://www.onsemi.com/products/discrete-power-modules/silicon-carbide-sic/silicon-carbide-sic-mosfets",
        "https://translate.google.com/translate?u=https://www.onsemi.com/products/discrete-power-modules/

2025-05-04 03:03:15,286 - INFO - 🚀 Starting task: 
```md
**Роль:** Ты — Автоматизированный Веб-Агент для Поиска Страниц.

**Цель:**  
Найти **все ссылки** из первой страницы поиска по запросу `Access Control / Key Systems (Safety)`.

**Инструкция:**

1. **Открой поисковую систему** (например, Google, Bing или аналогичную).

2. 
- **Введи запрос**: `Access Control / Key Systems (Safety)`.

3. **Из результатов поиска**:

   - Собери **все ссылки** на первой странице поиска.
   - Пропускай рекламные и спонсорские результаты.
   - Пропускай ссылки на социальные сети и видеохостинги.

4. **Фильтрация ссылок**:
   - Игнорируй все ссылки, в домене которых содержатся следующие ключевые слова:
     - `youtube`
     - `tiktok`
     - `facebook`
     - `instagram`
     - `twitter`
     - `linkedin`
     - `pinterest`
     - `vk`
     - `reddit`
     - `snapchat`
     - и аналогичные платформы социальных сетей или видеохостингов.
   - Проверку выполняй по части URL (поддомен и основной домен).
   


 -------------------------------- EXTRACTED DATA 
------------------------------------------

{
    "data": "{\n    \"text\": \"Access Control vs. Keys: Which Security System is Better?\",\n    \"href\": \"https://security.gallagher.com/en/Blog/access-control-vs-key-security\"\n  },\n  {\n    \"text\": \"\u05de\u05d9\u05d3\u05e2 \u05e2\u05dc \u05ea\u05e7\u05e6\u05d9\u05e8\u05d9 \u05ea\u05d5\u05e6\u05d0\u05d5\u05ea \u05d7\u05d9\u05e4\u05d5\u05e9 \u05e8\u05d0\u05e9\u05d5\u05e0\u05d5\u05ea\",\n    \"href\": \"/url?url=https://support.google.com/websearch?p%3Dfeatured_snippets%26hl%3Diw-IL&rct=j&q=&esrc=s&opi=89978449&usg=AOvVaw0vEHKqD-XCynuq-4MnEYc9&hl=iw&sa=X&ved=2ahUKEwjo3LyGwoiNAxUHSPEDHXNpAAwQrpwBegQIFhAC\"\n  },\n  {\n    \"text\": \"What Is Access Control? | Microsoft Security\",\n    \"href\": \"https://www.microsoft.com/en-us/security/business/security-101/what-is-access-control\"\n  },\n  {\n    \"text\": \"What Are The 3 Types of Access Control in Security?\",\n    \"href\": \"

2025-05-04 03:05:30,511 - INFO - 🚀 Starting task: 
```md
**Роль:** Ты — Автоматизированный Веб-Агент для Поиска Страниц.

**Цель:**  
Найти **все ссылки** из первой страницы поиска по запросу `Docking Connectors`.

**Инструкция:**

1. **Открой поисковую систему** (например, Google, Bing или аналогичную).

2. 
- **Введи запрос**: `Docking Connectors`.

3. **Из результатов поиска**:

   - Собери **все ссылки** на первой странице поиска.
   - Пропускай рекламные и спонсорские результаты.
   - Пропускай ссылки на социальные сети и видеохостинги.

4. **Фильтрация ссылок**:
   - Игнорируй все ссылки, в домене которых содержатся следующие ключевые слова:
     - `youtube`
     - `tiktok`
     - `facebook`
     - `instagram`
     - `twitter`
     - `linkedin`
     - `pinterest`
     - `vk`
     - `reddit`
     - `snapchat`
     - и аналогичные платформы социальных сетей или видеохостингов.
   - Проверку выполняй по части URL (поддомен и основной домен).
   - Если URL содержит хотя бы одно из эт